In [2]:
!pip install pytrends

  Preparing metadata (setup.py) ... done
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=fdfe4cdddab9a3f9f91f93583c78558024a9bc6b957ae263cb2830c05fc9a800
  Stored in directory: /root/.cache/pip/wheels/87/a9/a3/c190a7e0872639762e92cf3a84d6845ed086e10966533faf42
Successfully built pytrends
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
from pytrends.request import TrendReq
import pandas as pd
import sqlite3
import json
import urllib
import requests
from bs4 import BeautifulSoup
import time

# Country Codes

* PH-14	Autonomous Region in Muslim Mindanao       ARMM
* PH-05	Bicol                                      V
* PH-02	Cagayan Valley                             II
* PH-40	Calabarzon	                               IV-A
* PH-13	Caraga	                                   XIII
* PH-03	Central Luzon	                           III
* PH-07	Central Visayas	                           VII
* PH-15	Cordillera Administrative Region	       CAR
* PH-11	Davao	                                   XI
* PH-08	Eastern Visayas	                           VIII
* PH-01	Ilocos	                                   I
* PH-41	Mimaropa	                               IV-B
* PH-00	National Capital Region	                   NCR
* PH-10	Northern Mindanao	                       X
* PH-12	Soccsksargen		                       XII
* PH-06	Western Visayas		                       VI
* PH-09	Zamboanga Peninsula	                       IX

# Payload

In [ ]:
pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=10, backoff_factor=.5)

### SQLite Database

In [ ]:
conn = sqlite3.connect('dengue_regions.db')
print("Opened database successfully");
conn.execute('DROP TABLE IF EXISTS dengue;')
conn.execute('''CREATE TABLE dengue
         (ID INT PRIMARY KEY,
         query TEXT,
         value INT,
         origin TEXT,
         type TEXT,
         region_code TEXT,
         region_name TEXT,
         timeframe TEXT);''')
print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


In [ ]:
# keyword and region list.
kw_list=[
'Dengue',
# 'Dengue Fever',
# 'Dengue Medicine',
# 'Dengue Cure',
# 'Dengue Symptoms',
]

regions = {
    # 'PH': 'Philippines',
    # 'PH-00':'National_Capital_Region_NCR',
    # 'PH-14':'Autonomous_Region_in_Muslim_Mindanao_ARMM',
    # 'PH-05':'Bicol_V',
    # 'PH-02':'Cagayan_Valley_II',
    # 'PH-40':'Calabarzon_IV_A',
    # 'PH-13':'Caraga_XIII',
    # 'PH-03':'Central_Luzon_III',
    'PH-07':'Central_Visayas_VII',
    # 'PH-15':'Cordillera_Administrative_Region_CAR',
    # 'PH-11':'Davao_XI',
    # 'PH-08':'Eastern_Visayas_VIII',
    # 'PH-01':'Ilocos_I',
    # 'PH-41':'Mimaropa_IV-B',
    # 'PH-10':'Northern_Mindanao_X',
    # 'PH-12':'Soccsksargen_XII',
    # 'PH-06':'Western_Visayas_VI',
    'PH-09':'Zamboanga_Peninsula_IX'
}

timeframes = [    
    '2012-12-31 2013-01-31',
    '2013-01-31 2013-02-28',
    '2013-02-28 2013-03-31',
    '2013-03-31 2013-04-30',
    '2013-04-30 2013-05-31',
    '2013-05-31 2013-06-30',
    '2013-06-30 2013-07-31',
    '2013-07-31 2013-08-31',
    '2013-08-31 2013-09-30',
    '2013-09-30 2013-10-31',
    '2013-10-31 2013-11-30',
    '2013-11-30 2013-12-31',
    '2013-12-31 2014-01-31',
    '2014-01-31 2014-02-28',
    '2014-02-28 2014-03-31',
    '2014-03-31 2014-04-30',
    '2014-04-30 2014-05-31',
    '2014-05-31 2014-06-30',
    '2014-06-30 2014-07-31',
    '2014-07-31 2014-08-31',
    '2014-08-31 2014-09-30',
    '2014-09-30 2014-10-31',
    '2014-10-31 2014-11-30',
    '2014-11-30 2014-12-31',
    '2014-12-31 2015-01-31',
    '2015-01-31 2015-02-28',
    '2015-02-28 2015-03-31',
    '2015-03-31 2015-04-30',
    '2015-04-30 2015-05-31',
    '2015-05-31 2015-06-30',
    '2015-06-30 2015-07-31',
    '2015-07-31 2015-08-31',
    '2015-08-31 2015-09-30',
    '2015-09-30 2015-10-31',
    '2015-10-31 2015-11-30',
    '2015-11-30 2015-12-31',
    '2015-12-31 2016-01-31',
    '2016-01-31 2016-02-28',
    '2016-02-28 2016-03-31',
    '2016-03-31 2016-04-30',
    '2016-04-30 2016-05-31',
    '2016-05-31 2016-06-30',
    '2016-06-30 2016-07-31',
    '2016-07-31 2016-08-31',
    '2016-08-31 2016-09-30',
    '2016-09-30 2016-10-31',
    '2016-10-31 2016-11-30',
    '2016-11-30 2016-12-31',
    '2016-12-31 2017-01-31',
    '2017-01-31 2017-02-28',
    '2017-02-28 2017-03-31',
    '2017-03-31 2017-04-30',
    '2017-04-30 2017-05-31',
    '2017-05-31 2017-06-30',
    '2017-06-30 2017-07-31',
    '2017-07-31 2017-08-31',
    '2017-08-31 2017-09-30',
    '2017-09-30 2017-10-31',
    '2017-10-31 2017-11-30',
    '2017-11-30 2017-12-31',
    '2017-12-31 2018-01-31',
    '2018-01-31 2018-02-28',
    '2018-02-28 2018-03-31',
    '2018-03-31 2018-04-30',
    '2018-04-30 2018-05-31',
    '2018-05-31 2018-06-30',
    '2018-06-30 2018-07-31',
    '2018-07-31 2018-08-31',
    '2018-08-31 2018-09-30',
    '2018-09-30 2018-10-31',
    '2018-10-31 2018-11-30',
    '2018-11-30 2018-12-31',
    '2018-12-31 2019-01-31',
    '2019-01-31 2019-02-28',
    '2019-02-28 2019-03-31',
    '2019-03-31 2019-04-30',
    '2019-04-30 2019-05-31',
    '2019-05-31 2019-06-30',
    '2019-06-30 2019-07-31',
    '2019-07-31 2019-08-31',
    '2019-08-31 2019-09-30',
    '2019-09-30 2019-10-31',
    '2019-10-31 2019-11-30',
    '2019-11-30 2019-12-31',
    '2019-12-31 2020-01-31',
    '2020-01-31 2020-02-28',
    '2020-02-28 2020-03-31',
    '2020-03-31 2020-04-30',
    '2020-04-30 2020-05-31',
    '2020-05-31 2020-06-30',
    '2020-06-30 2020-07-31',
    '2020-07-31 2020-08-31',
    '2020-08-31 2020-09-30',
    '2020-09-30 2020-10-31',
    '2020-10-31 2020-11-30',
    '2020-11-30 2020-12-31',
    '2020-12-31 2021-01-31',
    '2021-01-31 2021-02-28',
    '2021-02-28 2021-03-31',
    '2021-03-31 2021-04-30',
    '2021-04-30 2021-05-31',
    '2021-05-31 2021-06-30',
    '2021-06-30 2021-07-31',
    '2021-07-31 2021-08-31',
    '2021-08-31 2021-09-30',
    '2021-09-30 2021-10-31',
    '2021-10-31 2021-11-30',
    '2021-11-30 2021-12-31',
    '2021-12-31 2022-01-31',
    '2022-01-31 2022-02-28',
    '2022-02-28 2022-03-31',
    '2022-03-31 2022-04-30',
    '2022-04-30 2022-05-31',
    '2022-05-31 2022-06-30',
    '2022-06-30 2022-07-31',
    '2022-07-31 2022-08-31',
]

# Google Trends Extraction with sleep of 5 seconds

In [ ]:
for idx,keyword in enumerate(kw_list):
    for region_code, region_name in regions.items():
        for timeframe in timeframes:
            print(idx,keyword, region_name, timeframe)
            
            pytrends.build_payload([keyword], timeframe=timeframe,geo=region_code, gprop='')
            
            orig_trends = pytrends.related_queries()
            conn = sqlite3.connect('dengue.db')
            if (orig_trends[keyword]['top'] is not None):
                top_df = pd.DataFrame(orig_trends[keyword]['top'])
                top_df['origin'] = keyword
                top_df['type'] = 'top'
                top_df['region_code'] = region_code
                top_df['region_name'] = region_name
                top_df['timeframe'] = timeframe
                top_df.to_sql('dengue_regions', conn, if_exists='append', index = False)
            else:
                print("Dataframe (top) empty for ", keyword, region_name, timeframe)

            if (orig_trends[keyword]['rising'] is not None):
                rising_df = pd.DataFrame(orig_trends[keyword]['rising'])
                rising_df['origin'] = keyword
                rising_df['type'] = 'rising'
                rising_df['region_code'] = region_code
                rising_df['region_name'] = region_name
                rising_df['timeframe'] = timeframe
                rising_df.to_sql('dengue_regions', conn, if_exists='append', index = False)
            else:
                print("Dataframe (rising) empty for ", keyword, region_name, timeframe)
            
            time.sleep(5)

0 Dengue Central_Visayas_VII 2012-12-31 2013-01-31
0 Dengue Central_Visayas_VII 2013-01-31 2013-02-28
Dataframe (rising) empty for  Dengue Central_Visayas_VII 2013-01-31 2013-02-28
0 Dengue Central_Visayas_VII 2013-02-28 2013-03-31
Dataframe (rising) empty for  Dengue Central_Visayas_VII 2013-02-28 2013-03-31
0 Dengue Central_Visayas_VII 2013-03-31 2013-04-30
Dataframe (top) empty for  Dengue Central_Visayas_VII 2013-03-31 2013-04-30
Dataframe (rising) empty for  Dengue Central_Visayas_VII 2013-03-31 2013-04-30
0 Dengue Central_Visayas_VII 2013-04-30 2013-05-31
0 Dengue Central_Visayas_VII 2013-05-31 2013-06-30
Dataframe (rising) empty for  Dengue Central_Visayas_VII 2013-05-31 2013-06-30
0 Dengue Central_Visayas_VII 2013-06-30 2013-07-31
Dataframe (rising) empty for  Dengue Central_Visayas_VII 2013-06-30 2013-07-31
0 Dengue Central_Visayas_VII 2013-07-31 2013-08-31
0 Dengue Central_Visayas_VII 2013-08-31 2013-09-30
0 Dengue Central_Visayas_VII 2013-09-30 2013-10-31
0 Dengue Central_Vi

# Extract data from SQLite into DF

In [4]:
conn = None
try:
    conn = sqlite3.connect('dengue.db')
except Error as e:
        print(e)

related_queries_df = pd.read_sql_query("SELECT * from dengue_regions", conn)
related_queries_df.to_csv("related_queries.csv")


In [ ]:
related_queries_df

,query,value,origin,type,region_code,region_name,timeframe
0,dengue symptoms,100,Dengue,top,PH,Philippines,2012-12-31 2013-01-31
1,dengue fever,94,Dengue,top,PH,Philippines,2012-12-31 2013-01-31
2,symptoms of dengue,51,Dengue,top,PH,Philippines,2012-12-31 2013-01-31
3,dengue fever symptoms,22,Dengue,top,PH,Philippines,2012-12-31 2013-01-31
4,signs of dengue,17,Dengue,top,PH,Philippines,2012-12-31 2013-01-31
...,...,...,...,...,...,...,...
4743,symptoms of dengue,100,Dengue Symptoms,top,PH-05,Bicol_V,2019-07-31 2019-08-31
4744,symptoms of dengue,100,Dengue Symptoms,top,PH-05,Bicol_V,2019-08-31 2019-09-30
4745,symptoms of dengue,100,Dengue Symptoms,top,PH-05,Bicol_V,2019-09-30 2019-10-31
4746,symptoms of dengue,100,Dengue Symptoms,top,PH-05,Bicol_V,2019-12-31 2020-01-31


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=abae83a8-7a4b-42fe-9d60-1f8de28a56c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>